<a href="https://colab.research.google.com/github/SY-256/causal-inference/blob/main/IPW%E3%81%AB%E3%82%88%E3%82%8BATE%E3%81%AE%E6%8E%A8%E5%AE%9A.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install causallib

In [ ]:
import causallib
import pandas as pd
import numpy as np
from causallib.datasets import load_nhefs
from causallib.evaluation import evaluate
from causallib.estimation import IPW
from sklearn.linear_model import LogisticRegression

In [ ]:
nhefs_data = load_nhefs()

In [ ]:
nhefs_data

In [ ]:
X = nhefs_data.X
a = nhefs_data.a
y = nhefs_data.y

In [ ]:
# 交絡因子
X

In [ ]:
# 禁煙をしたかどうか（した=1/していない=0）
a

In [ ]:
# 初期からの体重増加量
y

In [ ]:
df = pd.concat([X, a, y], axis=1)

In [ ]:
df

In [ ]:
df = df.rename(columns={
    "age^2": "age_sq",
    "wt71^2": "wt71_sq",
    "smokeintensity^2": "smokeintensity_sq",
    "smokeyrs^2": "smokeyrs_sq"
})

In [ ]:
df

In [ ]:
# ナイーブ推定量を計算(禁煙／喫煙での体重の変換の平均)
df_agg = df.groupby("qsmk")["wt82_71"].mean()
df_agg

In [ ]:
# 差分からナイーブ推定量を計算
# セレクションバイアスを含んでいる値
df_agg.loc[1] - df_agg[0]

In [ ]:
# 傾向スコアの算出にロジスティック回帰を使う
ipw = IPW(LogisticRegression())
ipw.fit(X, a, y)

In [ ]:
# 傾向スコアの確認
results = evaluate(ipw, X, a, y)

In [ ]:
# 傾向スコアの可視化プロット
# 傾向スコアが極端に小さい値を取っていないか
# treatment=1とtreatment=0がマッチした結果になっているか
results.plot_weight_distribution()

In [ ]:
outcomes = ipw.estimate_population_outcome(X, a, y)

In [ ]:
ate = ipw.estimate_effect(outcomes[1], outcomes[0])

In [ ]:
# 2.5の値から差がある⇒セレクションバイアスの影響
ate